In [11]:
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

In [12]:
# NHẬP - XUẤT - TỒN & EXCHANGE & GIÁ VỐN & MUA HÀNG & BÁN HÀNG

data_nhapxuatton=pd.read_excel("../VACP/Export BRAVO/Tổng hợp Nhập - Xuất - Tồn YTD.xls",
                        usecols="A:B,G:O",skipfooter=1,header=5).rename(columns={
                                "Unnamed: 0":"ID",
                                "Unnamed: 1":"Tên vật tư",
                                "Số lượng":"Tổng đầu kỳ", 
                                "Giá trị":"Giá trị đầu kỳ",
                                "Số lượng.1":"Tổng nhập", 
                                "Giá trị.1":"Giá trị nhập",
                                "Số lượng.2":"Tổng xuất", 
                                "Giá trị.2":"Giá trị xuất",
                                "Số lượng.3":"Tổng tồn", 
                                "Giá trị.3":"Giá trị tồn" })


data_muahang=pd.read_excel("../VACP/Export BRAVO/Sổ chi tiết mua hàng (2).xls",
                                usecols="A,C,F,I:J",skipfooter=1,header=5).rename(columns={
                                "Unnamed: 0":"Ngày nhập",
                                "Unnamed: 2":"Nhà cung cấp",
                                "Unnamed: 5":"ID",
                                "Số lượng":"Tổng nhập", 
                                "Đơn giá":"Giá nhập"
                                })

data_banhang=pd.read_excel("../VACP/Export BRAVO/Sổ chi tiết bán hàng.xls",
                        usecols="A,C:D,F,H,K:M",skipfooter=1,header=6).rename(columns={
                                "Unnamed: 0":"Ngày bán",
                                "Unnamed: 2":"ID Khách hàng",
                                "Unnamed: 3":"Hãng",
                                "Unnamed: 5":"ID",
                                "Unnamed: 7":"Tên hàng",
                                "Số lượng":"Tổng bán", 
                                "Đơn giá gốc":"Giá bán",
                                "Giá bán":"Giá bán giảm"
                                })



C:\Users\TAM\AppData\Local\Temp\ipykernel_14964\73445924.py:3: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  data_nhapxuatton=pd.read_excel("../VACP/Export BRAVO/Tổng hợp Nhập - Xuất - Tồn YTD.xls",


In [13]:
data_input=pd.read_excel("../VACP/Python NCC Min/Analysing Price 10 10/INPUT.xlsx"
                                ,usecols="B:M")

C:\Users\TAM\AppData\Local\Temp\ipykernel_14964\3960225145.py:1: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  data_input=pd.read_excel("../VACP/Python NCC Min/Analysing Price 10 10/INPUT.xlsx"


In [14]:
# Tính UNIT PRICE VND
data_exchange=pd.read_excel("../VACP/Export BRAVO/Vietcombank Exchange Currency.xlsx")
data_input=data_input.merge(data_exchange[["Mã nguyên tệ","Tỷ giá VND"]],on="Mã nguyên tệ")

#Chi phí chuyển đổi
ExchangeCost=0.05
data_input["Unit price VND"] = data_input["Unit price"]*data_input["Tỷ giá VND"]*(1+ExchangeCost)
data_input.head()

# Tính Unit Price Min
min_unit_price=data_input.groupby(["ID"])["Unit price VND"].min()
data_input_min=data_input.merge(min_unit_price,on="ID",how="left",suffixes=('', '_min'))
data_input_min=data_input_min[data_input_min["Unit price VND"]==data_input_min["Unit price VND_min"]].drop('Unit price VND_min',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)

In [15]:
# Giá vốn
TransCost=0.05
data_nhapxuatton['Giá vốn'] =(1+TransCost)*np.where(data_nhapxuatton["Giá trị xuất"]>0,
                                        data_nhapxuatton["Giá trị xuất"]/data_nhapxuatton["Tổng xuất"],
                                            np.where(data_nhapxuatton["Giá trị tồn"]>0,
                                                data_nhapxuatton["Giá trị tồn"]/data_nhapxuatton["Tổng tồn"],0

                                            )
)


In [16]:
#Mua hàng giá nhập nhỏ nhất
min_mua_hang=data_muahang.groupby(["ID"])["Giá nhập"].min()
data_muahang_min=data_muahang.merge(min_mua_hang,on="ID",how="left",suffixes=('', '_min'))
data_muahang_min=data_muahang_min[data_muahang_min["Giá nhập"]==data_muahang_min["Giá nhập_min"]].drop('Giá nhập_min',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)

In [17]:
#Bán hàng gần đây
max_date_banhang=data_banhang.groupby(["ID"])["Ngày bán"].max()
data_banhang_latest=data_banhang.merge(max_date_banhang,on="ID",how="left",suffixes=('', '_max'))
data_banhang_latest=data_banhang_latest[data_banhang_latest["Ngày bán"]==data_banhang_latest["Ngày bán_max"]].drop('Ngày bán_max',axis=1).drop_duplicates(subset="ID",keep='first').reset_index(drop=True)



In [18]:
# Gộp dữ liệu Nhập Xuất Tồn + MUA + BÁN
data_nhap_mua_ban=data_nhapxuatton.merge(data_muahang_min,on="ID").merge(data_banhang_latest,on="ID")
data_nhap_mua_ban_filter=data_nhap_mua_ban[["ID","Giá vốn","Ngày nhập","Giá nhập","Nhà cung cấp","Giá bán giảm","Tổng nhập_x","Tổng xuất", "Tổng tồn"]]


In [19]:
data_report=data_input_min.merge(data_nhap_mua_ban_filter, on="ID",how="left").drop_duplicates(subset=['ID','Nhà sản xuất']).reset_index(drop=True)
data_report

,ID,ID Item,Description,PCS/PACK,Exists Stocks,Qty request,Unit price,Mã nguyên tệ,Nhà sản xuất,Số lượng cần nhập,Tỷ giá VND,Unit price VND,Giá vốn,Ngày nhập,Giá nhập,Nhà cung cấp,Giá bán giảm,Tổng nhập_x,Tổng xuất,Tổng tồn
0,90080-38070,BUSH,NaN,NaN,NaN,NaN,0.390000,USD,NK-AST,NaN,24020.0,9.836190e+03,10282.539750,2020-02-12,8000.0,HTAUTOHN,30000.0,900.0,800.0,402.0
1,90080-38088,BUSH,NaN,NaN,NaN,NaN,0.390000,USD,NK-AST,NaN,24020.0,9.836190e+03,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,9004A-31023,"SEAL,VALVE STEM OIL",NaN,NaN,NaN,NaN,0.470000,USD,NK-AST,NaN,24020.0,1.185387e+04,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,90919-01059,"PLUG, SPARK",KF80 = KIJANG MINIBUS,NaN,NaN,NaN,0.803754,USD,NK-AMC,NaN,24020.0,2.027149e+04,19175.889983,2022-04-21,17265.0,NK-TRISTAN,30000.0,2214.0,2326.0,467.0
4,90919-01131-8N,PLUG Q20R-U11,NaN,NaN,NaN,NaN,0.820000,USD,NK-AST,NaN,24020.0,2.068122e+04,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,43430-0K070,"SHAFT ASSY, FR DRIVE",NaN,NaN,NaN,NaN,282.090000,USD,NK-AST,NaN,24020.0,7.114592e+06,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
383,81110-0KB80,"HEADLAMP ASSY, RH",NaN,NaN,NaN,NaN,295.290000,USD,NK-AST,NaN,24020.0,7.447509e+06,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
384,81150-0KB80,"HEADLAMP ASSY, LH",NaN,NaN,NaN,NaN,295.290000,USD,NK-AST,NaN,24020.0,7.447509e+06,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
385,81550-0K430,"LAMP ASSY, RR COMBINATION, RH",NaN,NaN,NaN,NaN,304.640000,USD,NK-AST,NaN,24020.0,7.683325e+06,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Python NCC Min/Analysing Price 10 10/Export_data.xlsx') as writer:
    data_nhap_mua_ban_filter.to_excel(writer, sheet_name='Nhập Mua Bán')
    data_report.to_excel(writer, sheet_name='Report')

